In [1]:
import mlrun

In [2]:
project = mlrun.get_or_create_project("great-expectations", context="./")

> 2022-03-17 17:08:01,600 [info] loaded project great-expectations from MLRun DB


### Create and Build Validation Function

In [12]:
fn = mlrun.code_to_function(
    name="validate_expectations",
    filename="great_expectations_mlrun.py",
    kind="job",
    handler="validate_expectations",
    requirements="requirements.txt",
    image="mlrun/mlrun",
    with_doc=False,
    #image=".mlrun/func-great-expectations-validate-expectations:latest"
).apply(mlrun.auto_mount())

In [13]:
fn.export()

> 2022-03-17 17:26:38,599 [info] function spec saved to path: function.yaml


In [7]:
fn.deploy()

> 2022-03-17 17:09:18,676 [info] Started building image: .mlrun/func-great-expectations-validate-expectations:latest
INFO[0000] Retrieving image manifest mlrun/mlrun:0.10.0 
INFO[0000] Retrieving image mlrun/mlrun:0.10.0 from registry index.docker.io 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest mlrun/mlrun:0.10.0 
INFO[0000] Returning cached image manifest              
INFO[0000] Executing 0 build triggers                   
INFO[0000] Unpacking rootfs as cmd RUN python -m pip install great-expectations==0.14.9 requires it. 
INFO[0019] RUN python -m pip install great-expectations==0.14.9 
INFO[0019] Taking snapshot of full filesystem...        
INFO[0037] cmd: /bin/sh                                 
INFO[0037] args: [-c python -m pip install great-expectations==0.14.9] 
INFO[0037] Running: [/bin/sh -c python -m pip install great-expectations==0.14.9] 
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.7
 

True

### Run Validation

In [8]:
run = fn.run(
    inputs={"data": "https://s3.wasabisys.com/iguazio/data/iris/iris.data.raw.csv"},
    params={
        "expectation_suite_name": "test_suite",
        "data_asset_name": "iris_dataset",
    },
)

> 2022-03-17 17:10:11,302 [info] starting run validate-expectations-validate_expectations uid=aac2d7327f474b5da6b778acbfa891d1 DB=http://mlrun-api:8080
> 2022-03-17 17:10:11,477 [info] Job is running in the background, pod: validate-expectations-validate-expectations-9mm7q
> 2022-03-17 17:10:21,909 [info] run executed, status=completed
Calculating Metrics: 100%|██████████| 15/15 [00:00<00:00, 238.11it/s]
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
great-expectations,...bfa891d1,0,Mar 17 17:10:18,completed,validate-expectations-validate_expectations,v3io_user=nickkind=jobowner=nickmlrun/client_version=0.10.0host=validate-expectations-validate-expectations-9mm7q,data,expectation_suite_name=test_suitedata_asset_name=iris_dataset,validated=False,validation_results


> 2022-03-17 17:10:30,904 [info] run executed, status=completed


### View Data Doc

In [9]:
import os
from IPython.display import IFrame

In [10]:
IFrame(src=os.path.relpath(run.outputs["validation_results"]), width=1000, height=800)